In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
student = os.path.join("Raw Data", "students_complete.csv")
school = os.path.join("Raw Data", "schools_complete.csv")
student_df=pd.read_csv(student)
school_df=pd.read_csv(school)
#student_df.head()
    #school_df.head()
#student_df.count() ##39170 rows
    #school_df.count() ##15 rows
#student_df.columns ##'Student ID', 'name', 'gender', 'grade', 'school', 'reading_score', 'math_score'
    #school_df.columns ##'School ID', 'name', 'type', 'size', 'budget'
    #school_df['name'].value_counts()

In [3]:
#Renaming the common index on both csv to 'School_Name' to enable join'
student_df1 = student_df.rename(columns={'school':'School_Name'})
school_df1 = school_df.rename(columns={'name':'School_Name'})

In [4]:
#Join tables, so that the school info is associated for all students
table_df = pd.merge(student_df1, school_df1, on="School_Name", how="left")
#merge_table.head()
#merge_table.columns

In [5]:
#####################################################District Summary########################################################

    #Create a high level snapshot (in table form) of the district's key metrics, including:
print("District Data Summary")
print("-------------------------------------------------------------------------------------------")    
    # Total Schools
print("Total Number of Schools: " + str(len(table_df['School_Name'].unique())))
#t_school=table_df['School_Name'].unique() ##Method 2 by creating a new variable
#len(t_school)
    
      # Total Students
print("Total Number of Students: " + str(len(table_df['Student ID'].unique())))
#>>>39170 which corresponds to the total number of rows, meaning also that there is not either a single student or two students with the same ID enrolled in two schools :D
      
    # Total Budget
print("Total Budget: $" + str(table_df['budget'].unique().sum()))
        ##Method 2 by creating a new variable;   #budget = table_df['budget'].unique() 
                                                #budget.sum()
      # Average Math Score
print("Average Math Score: " + str(table_df['math_score'].mean()))
      #>>> 78.98537145774827

      # Average Reading Score
print("Average Reading Score: " + str(table_df['reading_score'].mean()))
      #>>> 81.87784018381414
      
      # % Passing Math
print("% Passing Math: " + str(len(table_df.loc[table_df["math_score"] >= 70,:]) /  len(table_df['Student ID'].unique())))
math=len(table_df.loc[table_df["math_score"] >= 70,:]) / len(table_df['Student ID'].unique())
    #>>>0.749808526933878
    
    # % Passing Reading
print("% Passing Math: " + str(len(table_df.loc[table_df["reading_score"] >= 70,:]) /  len(table_df['Student ID'].unique())))
reading=len(table_df.loc[table_df["reading_score"] >= 70,:]) /  len(table_df['Student ID'].unique())
    #>>>0.8580546336482001
    
      # Overall Passing Rate (Average of the above two)
print("% Average Passing Rate: " + str((math+reading)/2))

District Data Summary
-------------------------------------------------------------------------------------------
Total Number of Schools: 15
Total Number of Students: 39170
Total Budget: $24649428
Average Math Score: 78.98537145774827
Average Reading Score: 81.87784018381414
% Passing Math: 0.749808526933878
% Passing Math: 0.8580546336482001
% Average Passing Rate: 0.8039315802910391


In [6]:
###############################################################################################################################

In [6]:
########################################################School Summary########################################################
#Filter main table attributes
table_dfl=table_df.loc[:,["School_Name","type", "size", "budget", "math_score", "reading_score"]]

table_dfl #retains 39170 rows
#table_df2=table_dfl.groupby("School_Name")
#table_df2.head() #75 rows, why???? Doesn't even group properly
r70=table_dfl.loc[table_dfl["reading_score"] >= 70,:] #Filters properly
m70=table_dfl.loc[table_dfl["math_score"] >= 70,:]
#count of students who passed per school / size
r70=r70.groupby("School_Name").count() #counts all who passed per school. Cool, now isolate one row.
m70=m70.groupby("School_Name").count()
r70=r70["reading_score"] #choosing just one row. All values are the same per rows, since each row represent a student who passed.
m70=m70["math_score"]

table_df1=table_dfl.groupby("School_Name").mean() #Averages of the tests scores by schools
table_df1_type=table_dfl.groupby("School_Name")
table_df1_type=table_df1_type['type']

In [7]:
#school_sum_df = table_df.loc[:,["School_Name","type", "size", "budget", "math_score", "reading_score"]]
#x=school_sum_df.groupby("School_Name")

#Creating my variables for the summary table dataframe
size=table_df1["size"].unique() #Is fine, unique attribute
pm=m70/size #Passing rate, math
rm=r70/size #Passing rate, reading
stype=table_df1_type.unique() #Is fine, unique attribute
budget=table_df1["budget"].unique() #Is fine, unique attribute
math_score=table_df1["math_score"] #"table_df1.head()" outputs 75 rows; however the avg numbers checks out
reading_score=table_df1["reading_score"] #"table_df1.head()" outputs 75 rows; however the avg numbers checks out
##method 2 to gather averages:
#av_r=scores["reading_score"] #avg math score per school
#av_m=scores["math_score"] #avg reading score per school
psb = table_df1["budget"].unique() / table_df1["size"].unique() #Error prob contigent on prior variables
#o_avg = (table_df1["math_score"].mean() + table_df1["reading_score"].mean())/2 #That's actually the avg overall score
opr = (pm + rm) /2 #got it :) !

In [8]:
school_summary_table = pd.DataFrame({"Type":stype,
                                    "Total_Students":size,
                                     "Total_School_Budget":budget,
                                     "Per_Student_Budget":psb,
                                     "Average_Math_Score":math_score,
                                     "Average_Reading_Score":reading_score,
                                     "Passing_Math":pm*100,
                                     "Passing_Reading":rm*100,
                                   "Overall_Passing_Rate":opr*100})

#mapping (cleaning up how values are displayed)
school_summary_table["Total_School_Budget"] = school_summary_table["Total_School_Budget"].map("${:,.0f}".format)
school_summary_table["Per_Student_Budget"] = school_summary_table["Per_Student_Budget"].map("${:,.0f}".format)
school_summary_table["Total_Students"]=school_summary_table["Total_Students"].map("{:,.0f}".format)
school_summary_table["Passing_Math"]=school_summary_table["Passing_Math"].map("{:.2f}%".format)
school_summary_table["Passing_Reading"]=school_summary_table["Passing_Reading"].map("{:.2f}%".format)
school_summary_table["Overall_Passing_Rate"]=school_summary_table["Overall_Passing_Rate"].map("{:.2f}%".format)
school_summary_table["Average_Math_Score"]=school_summary_table["Average_Math_Score"].map("{:,.2f}".format)
school_summary_table["Average_Reading_Score"]=school_summary_table["Average_Reading_Score"].map("{:,.2f}".format)

school_summary_table.head(15)

,Average_Math_Score,Average_Reading_Score,Overall_Passing_Rate,Passing_Math,Passing_Reading,Per_Student_Budget,Total_School_Budget,Total_Students,Type
School_Name,,,,,,,,,
Bailey High School,77.05,81.03,74.31%,66.68%,81.93%,$628,"$3,124,928","4,976",[District]
Cabrera High School,83.06,83.98,95.59%,94.13%,97.04%,$582,"$1,081,356","1,858",[Charter]
Figueroa High School,76.71,81.16,73.36%,65.99%,80.74%,$639,"$1,884,411","2,949",[District]
Ford High School,77.10,80.75,73.80%,68.31%,79.30%,$644,"$1,763,916","2,739",[District]
Griffin High School,83.35,83.82,95.27%,93.39%,97.14%,$625,"$917,500","1,468",[Charter]
Hernandez High School,77.29,80.93,73.81%,66.75%,80.86%,$652,"$3,022,020","4,635",[District]
Holden High School,83.80,83.81,94.38%,92.51%,96.25%,$581,"$248,087",427,[Charter]
Huang High School,76.63,81.18,73.50%,65.68%,81.32%,$655,"$1,910,635","2,917",[District]
Johnson High School,77.07,80.97,73.64%,66.06%,81.22%,$650,"$3,094,650","4,761",[District]


In [12]:
#mapping columns (work in progress)

#school_summary_table["Total School Budget"]=school_summary_table["Total School Budget"].map("${:.2f}".format)
#school_summary_table["Total Students"]=school_summary_table["Total Students"].map("{:,}".format)
#school_summary_table.dtypes

In [13]:
###############################################################################################################################

In [14]:
#######################################Top Performing Schools (By Passing Rate)################################################

In [9]:
top_schools=school_summary_table.sort_values("Overall_Passing_Rate", ascending=False)
top_schools.head(5)

,Average_Math_Score,Average_Reading_Score,Overall_Passing_Rate,Passing_Math,Passing_Reading,Per_Student_Budget,Total_School_Budget,Total_Students,Type
School_Name,,,,,,,,,
Cabrera High School,83.06,83.98,95.59%,94.13%,97.04%,$582,"$1,081,356","1,858",[Charter]
Thomas High School,83.42,83.85,95.29%,93.27%,97.31%,$638,"$1,043,130","1,635",[Charter]
Griffin High School,83.35,83.82,95.27%,93.39%,97.14%,$625,"$917,500","1,468",[Charter]
Pena High School,83.84,84.04,95.27%,94.59%,95.95%,$609,"$585,858",962,[Charter]
Wilson High School,83.27,83.99,95.20%,93.87%,96.54%,$578,"$1,319,574","2,283",[Charter]


In [136]:
#######################################Low Performing Schools (By Passing Rate)#############################################

In [10]:
top_schools=school_summary_table.sort_values("Overall_Passing_Rate", ascending=True)
top_schools.head(5)

,Average_Math_Score,Average_Reading_Score,Overall_Passing_Rate,Passing_Math,Passing_Reading,Per_Student_Budget,Total_School_Budget,Total_Students,Type
School_Name,,,,,,,,,
Rodriguez High School,76.84,80.74,73.29%,66.37%,80.22%,$637,"$2,547,363","3,999",[District]
Figueroa High School,76.71,81.16,73.36%,65.99%,80.74%,$639,"$1,884,411","2,949",[District]
Huang High School,76.63,81.18,73.50%,65.68%,81.32%,$655,"$1,910,635","2,917",[District]
Johnson High School,77.07,80.97,73.64%,66.06%,81.22%,$650,"$3,094,650","4,761",[District]
Ford High School,77.10,80.75,73.80%,68.31%,79.30%,$644,"$1,763,916","2,739",[District]


In [18]:
###############################################################################################################################

In [19]:
#################################################Average Scores by Grades#######################################################

In [11]:
#Creating the variables for Avg Reading score, per grade, per school
table_grade_read_df=table_df.loc[:,["reading_score","grade","School_Name"]]
table_grade_read_df.head()
#9th, Read
table_grade_df_9=table_grade_read_df[table_grade_read_df["grade"]=="9th"]
gr_avgread_9=table_grade_df_9.groupby(["School_Name"]).mean().reset_index()
#10th, Read
table_grade_df_10=table_grade_read_df[table_grade_read_df["grade"]=="10th"]
gr_avgread_10=table_grade_df_10.groupby(["School_Name"]).mean().reset_index() #Mean reading score for 10th graders, for each school
#11th, Read
table_grade_df_11=table_grade_read_df[table_grade_read_df["grade"]=="11th"]
gr_avgread_11=table_grade_df_11.groupby(["School_Name"]).mean().reset_index()
#12th, Read
table_grade_df_12=table_grade_read_df[table_grade_read_df["grade"]=="12th"]
gr_avgread_12=table_grade_df_12.groupby(["School_Name"]).mean().reset_index()

#Creating the variables for Avg Math score, per grade, per school
table_grade_math_df=table_df.loc[:,["math_score","grade","School_Name"]]
table_grade_math_df.head()
#9th, Math
table_grade_math_df_9=table_grade_math_df[table_grade_math_df["grade"]=="9th"]
gr_avgmath_9=table_grade_math_df_9.groupby(["School_Name"]).mean().reset_index()
#10th, Math
table_grade_math_df_10=table_grade_math_df[table_grade_math_df["grade"]=="10th"]
gr_avgmath_10=table_grade_math_df_10.groupby(["School_Name"]).mean().reset_index() 
#11th, Math
table_grade_math_df_11=table_grade_math_df[table_grade_math_df["grade"]=="11th"]
gr_avgmath_11=table_grade_math_df_11.groupby(["School_Name"]).mean().reset_index()
#12th, Math
table_grade_math_df_12=table_grade_math_df[table_grade_math_df["grade"]=="12th"]
gr_avgmath_12=table_grade_math_df_12.groupby(["School_Name"]).mean().reset_index()

In [12]:
#Merging Dataframes for Avg Math Scores per Grade & Schools because I couldn't be bothered with f#%$*ing creating a DataFrame that doesn't recognize the "School_Name" key despite being f&&*$ there
m_910=pd.merge(gr_avgmath_9,gr_avgmath_10,on='School_Name')
m_91011=pd.merge(m_910,gr_avgmath_11,on='School_Name')
m_finalmerge=pd.merge(m_91011,gr_avgmath_12,on='School_Name')
m_finalmerge.columns = ["School_Name","9th","10th","11th","12th"]

m_finalmerge["9th"]=m_finalmerge["9th"].map("{:,.2f}".format)
m_finalmerge["10th"]=m_finalmerge["10th"].map("{:,.2f}".format)
m_finalmerge["11th"]=m_finalmerge["11th"].map("{:,.2f}".format)
m_finalmerge["12th"]=m_finalmerge["12th"].map("{:,.2f}".format)

m_finalmerge
#m_finalmerge.rename(columns={'math_score_x':'9th','math_score_y':'10th','math_score_x':'11th','math_score_y':'12th'})

,School_Name,9th,10th,11th,12th
0,Bailey High School,77.08,77.00,77.52,76.49
1,Cabrera High School,83.09,83.15,82.77,83.28
2,Figueroa High School,76.40,76.54,76.88,77.15
3,Ford High School,77.36,77.67,76.92,76.18
4,Griffin High School,82.04,84.23,83.84,83.36
5,Hernandez High School,77.44,77.34,77.14,77.19
6,Holden High School,83.79,83.43,85.00,82.86
7,Huang High School,77.03,75.91,76.45,77.23
8,Johnson High School,77.19,76.69,77.49,76.86
9,Pena High School,83.63,83.37,84.33,84.12


In [140]:
#Merging Dataframes for Avg Reading Scores per Grade & Schools
r_910=pd.merge(gr_avgread_9,gr_avgread_10,on='School_Name')
r_91011=pd.merge(r_910,gr_avgread_11,on='School_Name')
r_finalmerge=pd.merge(r_91011,gr_avgread_12,on='School_Name')
r_finalmerge.columns = ["School_Name","9th","10th","11th","12th"]

r_finalmerge["9th"]=r_finalmerge["9th"].map("{:,.2f}".format)
r_finalmerge["10th"]=r_finalmerge["10th"].map("{:,.2f}".format)
r_finalmerge["11th"]=r_finalmerge["11th"].map("{:,.2f}".format)
r_finalmerge["12th"]=r_finalmerge["12th"].map("{:,.2f}".format)

r_finalmerge
#r_finalmerge=r_finalmerge.rename(columns={'reading_score_x':'9th','reading_score_y':'10th','reading_score_x':'11th','reading_score_y':'12th'})

,School_Name,9th,10th,11th,12th
0,Bailey High School,81.30,80.91,80.95,80.91
1,Cabrera High School,83.68,84.25,83.79,84.29
2,Figueroa High School,81.20,81.41,80.64,81.38
3,Ford High School,80.63,81.26,80.40,80.66
4,Griffin High School,83.37,83.71,84.29,84.01
5,Hernandez High School,80.87,80.66,81.40,80.86
6,Holden High School,83.68,83.32,83.82,84.70
7,Huang High School,81.29,81.51,81.42,80.31
8,Johnson High School,81.26,80.77,80.62,81.23
9,Pena High School,83.81,83.61,84.34,84.59


In [141]:
##Problems with combining DFs into one due to error pertaining to keys. Which is why I merged instead.

##.keys() I am losing the School_Name index somehow after grouping, which is why I can't combine into one dataframe.

#score_summary_read_grades=pd.DataFrame({"9th":gr_avgread_9,
#                                    "10th":gr_avgread_10,
#                                          "11th":gr_avgread_11,
#                                         "12th": gr_avgread_12
#                                     })


#Error: If using all scalar values, you must pass an index

#score_summary_math_grades=pd.DataFrame({"9th":gr_avgmath_9,
#                                    "10th":gr_avgmath_10,
#                                         "11th":gr_avgmath_11,
#                                         "12th": gr_avgmath_12
#                                    })

#merge_test=pd.merge(gr_avgread_9,gr_avgread_10,on='School_Name')
#merge_test.head()
#gr_avgread_9.head()

##Didn't Index correctly (previous attempt, original submission)

#table_m_gr=table_df.loc[:,["School_Name", "math_score","reading_score","grade"]]
#table_m_gr1=table_m_gr.groupby(['School_Name','grade'])
#m_gr=table_m_gr1["math_score"].mean()
#r_gr=table_m_gr1["reading_score"].mean()
#school_score_grades = pd.DataFrame({"Average Math Score":m_gr,
#                                    "Average Reading Score":r_gr,
#                                     })
#school_score_grades

In [143]:
###############################################################################################################################

In [130]:
budget_summary_table=pd.cut(table_df1["budget"].unique(), table_budget_bins, labels=labels_bin)
budget_summary_table = pd.DataFrame({"Budget_Range": budget_summary_table, #Need to recreated variables to index by bins
                                     "Average_Math_Score":math_score,
                                     "Average_Reading_Score":reading_score,
                                     "Passing_Reading":rm,
                                     "Passing_Math":pm,
                                   "Overall_Passing_Rate":opr
                                    })


# budget_summary_table["Budget_Range"] = budget_summary_table["Budget_Range"]
# budget_summary_table = budget_summary_table.set_index("Budget_Range")
#budget_summary_table = budget_summary_table.groupby("Budget_Range").mean()
budget_summary_table.head(15)

,Average_Math_Score,Average_Reading_Score,Budget_Range,Overall_Passing_Rate,Passing_Math,Passing_Reading
School_Name,,,,,,
Bailey High School,77.048432,81.033963,"=< $3,200,000",0.743067,0.666801,0.819333
Cabrera High School,83.061895,83.975780,"=< $1,500,000",0.955867,0.941335,0.970398
Figueroa High School,76.711767,81.158020,"=< $2,000,000",0.733639,0.659885,0.807392
Ford High School,77.102592,80.746258,"=< $2,000,000",0.738043,0.683096,0.792990
Griffin High School,83.351499,83.816757,"=< $1,000,000",0.952657,0.933924,0.971390
Hernandez High School,77.289752,80.934412,"=< $3,200,000",0.738080,0.667530,0.808630
Holden High School,83.803279,83.814988,"=< $1,000,000",0.943794,0.925059,0.962529
Huang High School,76.629414,81.182722,"=< $2,000,000",0.735002,0.656839,0.813164
Johnson High School,77.072464,80.966394,"=< $3,200,000",0.736400,0.660576,0.812224


In [172]:
###################################################Scores by School Spending##################################################
budget_summary_table=pd.cut(table_df1["budget"].unique(), table_budget_bins, labels=labels_bin)
budget_summary_table = pd.DataFrame({"Budget_Range": budget_summary_table, 
                                     "Average_Math_Score":math_score,
                                     "Average_Reading_Score":reading_score,
                                     "Passing_Reading":rm,
                                     "Passing_Math":pm,
                                   "Overall_Passing_Rate":opr
                                    })

#Observe budget data to create reasonable ranges for the bins
                                                                # budget.mean() #Just use the variable instead #>>>1643295.2
                                                                # budget.min() #248087
                                                                # budget.max() #3124928
table_budget_bins = [0,1000000,1500000,2000000,3200000]
labels_bin = ["=< $1,000,000","=< $1,500,000","=< $2,000,000","=< $3,200,000"]
labels_bins_code = [1,2,3,4]

budget_summary_table['Budget'] = budget
# budget_summary_table = budget_summary_table.groupby("Budget").mean()

budget_summary_table['Budget_Range'] = pd.cut(table_df1["budget"].unique(), table_budget_bins, labels=labels_bins_code)
budget_summary_table = budget_summary_table.set_index("Budget_Range")

budget_summary_table = budget_summary_table.groupby("Budget_Range",as_index=True).mean()

budget_summary_table["Overall_Passing_Rate"]=budget_summary_table["Overall_Passing_Rate"].map("{:.2f}%".format)
budget_summary_table["Passing_Math"]=budget_summary_table["Passing_Math"].map("{:.2f}%".format)
budget_summary_table["Passing_Reading"]=budget_summary_table["Passing_Reading"].map("{:.2f}%".format)
budget_summary_table["Average_Math_Score"]=budget_summary_table["Average_Math_Score"].map("{:,.2f}".format)
budget_summary_table["Average_Reading_Score"]=budget_summary_table["Average_Reading_Score"].map("{:,.2f}".format)

#budget_summary_table = budget_summary_table.replace({'NaN': "=< $1,000,000", 1.0: "=< $1,500,000", 2.0: "=< $2,000,000",3.0:"=< $3,200,000"})

budget_summary_table.head()
#print(budget_summary_table.dtypes)
###################################################################################################

,Average_Math_Score,Average_Reading_Score,Overall_Passing_Rate,Passing_Math,Passing_Reading,Budget
Budget_Range,,,,,,
NaN,83.66,83.89,0.95%,0.93%,0.96%,5.838150e+05
1.0,83.36,83.90,0.95%,0.94%,0.97%,1.110012e+06
2.0,76.81,81.03,0.74%,0.67%,0.80%,1.852987e+06
3.0,77.06,80.92,0.74%,0.66%,0.81%,2.947240e+06


In [29]:
###############################################################################################################################

In [30]:
###################################################Scores by School Size######################################################

In [31]:
school_summary_table["Total Students"].mean() #2611
school_summary_table["Total Students"].min() #427
school_summary_table["Total Students"].max() #4976

groups = ['Small (> 2,000)', 'Medium (2,000 - 3,000)', 'Large (3,000 - 5,000)']
bins = [400,2000,3000,5000]

KeyError: 'Total Students'

In [32]:
table_size_bins=pd.cut(school_summary_table["Total Students"], bins,labels=groups)
table_size_bins.head
size_summary_table = pd.DataFrame({"Size Range": table_size_bins,
                                     "Average Math Score":math_score,
                                     "Average Reading Score":reading_score,
                                     "% Passing Math":pm,
                                     "% Passing Reading":rm,
                                   "Overall Passing Rate":opr
                                    })
size_summary_table.head(15)

KeyError: 'Total Students'

In [33]:
###############################################################################################################################

In [34]:
###################################################Scores by School Type######################################################

In [35]:
#Prior functions (for reference)
table_dfl_type=table_df.loc[:,["type", "math_score", "reading_score"]]
table_dfl_type
table_dfl_math=table_df.loc[:,["type","math_score"]]
table_dfl_read=table_df.loc[:,["type","reading_score"]]

###################
t_scores=table_dfl_type.groupby("type").mean() #Avg for all my attributes selected above # !Cannot import into new Dataframe

t_math=t_scores["math_score"] #Mean for Math only * or...t_scores.math_score

t_read=t_scores["reading_score"] #Mean for Read only *

#Means of Scores ###REDUNDANT
t_70_pass=table_df.loc[:,["type","math_score", "reading_score"]]
t_mean=t_70_pass.groupby("type").mean() #Averages of BOTH tests scores by types # !Cannot import into new Dataframe


t_r70=table_dfl_read.loc[table_dfl_read["reading_score"] >= 70,:] #Filters properly
t_r70=t_r70.groupby("type").count() #Number of students who passed the reading test for BOTH school types *
t_m70=table_dfl_math.loc[table_dfl_math["math_score"] >= 70,:]
t_m70=t_m70.groupby("type").count()  #Number of students who passed the math test for BOTH school types *

#Percentage of students who passed the tests

total_count_read=table_df.loc[:,["type","reading_score"]]
total_students_read=total_count_read.groupby("type").count() #total students; the same number of students took the math and reading exams.
                                                        #I'm only doing this twice to be able to divide by appropriate columns
total_count_math=table_df.loc[:,["type","math_score"]]
total_students_math=total_count_math.groupby("type").count()

student_perc_pass_reading=t_r70/total_students_read
student_perc_pass_reading=student_perc_pass_reading["reading_score"] #Avg read score of those who passed *
student_perc_pass_math=t_m70/total_students_math
student_perc_pass_math=student_perc_pass_math["math_score"] #Avg math score of those who passed *

#Average Passing Rate
avg_passing_rate=(student_perc_pass_reading+student_perc_pass_math)/2 #Avg Passing Rate *

#Counts for total students in Charter and District schools ###Redundant
#charter_student_count=table_dfl_type[table_dfl_type["type"]=='Charter'].count()
#charter_student_count=charter_student_count["type"] #12194
#district_student_count=table_dfl_type[table_dfl_type["type"]=='District'].count()
#district_student_count=district_student_count["type"] #26976

In [36]:
district_summary_tableX = pd.DataFrame({"Average Math Scores":t_math,
                                        "Average Reading Scores":t_read,
                                    "% Passing Math":student_perc_pass_math,
                                    "% Passing Reading":student_perc_pass_reading,
                                    "% Overall Passing Rate": avg_passing_rate
                                       })
district_summary_tableX.head()

,% Overall Passing Rate,% Passing Math,% Passing Reading,Average Math Scores,Average Reading Scores
type,,,,,
Charter,0.951739,0.937018,0.966459,83.406183,83.902821
District,0.737118,0.665184,0.809052,76.987026,80.962485


In [37]:
#Count for students who passed, Charter & District ###Way Redundant, essentially if I want to isolate my variables by 
# score & district type. Keeping this here for reference.

#Charter, Passed Reading
#ctype_count_r70=r70[r70["type"]=='Charter'].count()
#ctype_count_r70=ctype_count_r70["type"] #>>> 11785

#Charter, Passed Math
#ctype_count_m70=m70[m70["type"]=='Charter'].count()
#ctype_count_m70=ctype_count_m70["type"] #>>> 11426

#District, Passed Reading
#dtype_count_r70=r70[r70["type"]=='District'].count()
#dtype_count_r70=dtype_count_r70["type"] #>>> 21825

#District, Passed Math
#dtype_count_m70=m70[m70["type"]=='District'].count()
#dtype_count_m70=dtype_count_m70["type"] #>>> 17944

#Total Count for students in Charter & District

#Total Students, Charter
#ctype_totalcount_r70=table_dfl[table_dfl["type"]=='Charter'].count()
#ctype_totalcount_r70=ctype_totalcount_r70["type"] #>>>12194

#ReadTotal Students, District
#dtype_totalcount_r70=table_dfl[table_dfl["type"]=='District'].count()
#dtype_totalcount_r70=dtype_totalcount_r70["type"] #>>>26976

#district_passed_reading= dtype_count_r70 / dtype_totalcount_r70 #0.8090524911032029
#district_passed_math= dtype_count_m70 / dtype_totalcount_r70 #0.6651838671411625
#charter_passed_reading= ctype_count_r70 / ctype_totalcount_r70 #0.9664589142201082
#charter_passed_math= ctype_count_m70 / ctype_totalcount_r70 #0.9370182056749221

In [32]:
###############################################################################################################################

In [ ]:
#Output pdf
    #<Insert function here>